Scraping [derby names](https://en.wikipedia.org/wiki/Roller_derby#Derby_names) from publicly-accessible lists

In [3]:
import os
import sys
import string
from pathlib import Path

from bs4 import BeautifulSoup
import requests

import pandas as pd
import advertools as adv
from tqdm import tqdm

# from tqdm.auto import tqdm

In [4]:
if "google.colab" in sys.modules:
    from google.colab import files
    from sklearn.utils import shuffle

In [5]:
session = requests.Session()

In [28]:
# Download the WFTDA list of certified officials
try:
    wftda_df = pd.DataFrame()
    url = "https://resources.wftda.org/officiating/roller-derby-certification-program-for-officials/roster-of-certified-officials/"
    print("Downloading names from {}".format(url))
    
    r = session.get(url=url)
    soup = BeautifulSoup(markup=r.text, features="lxml")
    rows = soup.find_all(name="h5")
    urls = [r.find("a")["href"] for r in rows]
    names = [r.find("a").get_text() for r in rows]
    wftda_df = pd.DataFrame(data={"Name": names, "url": urls})
except Exception as e:
    print(e)

In [29]:
if len(wftda_df) == 0:
    wftda_df = pd.read_csv(filepath_or_buffer="wftda.csv")
else:
    wftda_df.to_csv(path_or_buf="wftda.csv", index=False)
wftda_df

,Name,url
0,9mm Ram-Paige,https://resources.wftda.org/officiating/roller...
1,A. Grue,https://resources.wftda.org/officiating/roller...
2,A’Blazing Grace,https://resources.wftda.org/officiating/roller...
3,Adam Smasher,https://resources.wftda.org/officiating/roller...
4,Adam Splitter,https://resources.wftda.org/officiating/roller...
...,...,...
570,Yu Cypher,https://resources.wftda.org/officiating/roller...
571,Yvel Saint Laurent,https://resources.wftda.org/officiating/roller...
572,Zebra 3,https://resources.wftda.org/officiating/roller...
573,Zero,https://resources.wftda.org/officiating/roller...


In [34]:
# Download TwoEvils list of skaters
url = "https://www.twoevils.org/rollergirls/"
print("Downloading names from %s" % url)
twoevils_df = pd.DataFrame()

try:
    twoevils_df = pd.read_html(io=url, skiprows=1)[0]
    twoevils_df.columns = [h.replace("Skater", "").strip() for h in twoevils_df.iloc[0]]
    twoevils_df = twoevils_df.rename(columns={"Date Added": "Registered"})
    twoevils_df = twoevils_df.iloc[1:-1, :].dropna(how="all")
    twoevils_df["url"] = url
except Exception as e:
    print(e)

<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>


In [35]:
if len(twoevils_df) == 0:
    twoevils_df = pd.read_csv(filepath_or_buffer="twoevils.csv")
else:
    twoevils_df.to_csv(path_or_buf="twoevils.csv", index=False)
twoevils_df

FileNotFoundError: [Errno 2] No such file or directory: 'twoevils.csv'

In [36]:
# Download list from Derby Roll Call
try:
    drc_url =  "http://www.derbyrollcall.com/everyone"
    drc_df = pd.read_html(io=drc_url)[0].rename(columns={"#": "Number"}).dropna(subset=["Name"])
    drc_df["url"] = drc_url
except Exception as e:
    print(e)

In [37]:
if len(drc_df) == 0:
    drc_df = pd.read_csv(filepath_or_buffer="derbyrollcall.csv")
else:
    drc_df.to_csv(path_or_buf="derbyrollcall.csv", index=False)
drc_df

,Name,Number,League,Country,Registered,url
0,Sausage Roller,14,Manchester Roller Derby,United Kingdom,"1st January, 1970",http://www.derbyrollcall.com/everyone
1,James Mean,27,Manchester Roller Derby,United Kingdom,"20th January, 2014",http://www.derbyrollcall.com/everyone
2,Yvel Saint Laurent,14,Knights of Oldham Roller Derby & Rainy City Ro...,United Kingdom,"20th January, 2014",http://www.derbyrollcall.com/everyone
3,Pancake,1928,Rainy City Roller Derby,United Kingdom,"20th January, 2014",http://www.derbyrollcall.com/everyone
4,Robert Quadriguez,101,Crash Test Brummies,United Kingdom,"20th January, 2014",http://www.derbyrollcall.com/everyone
...,...,...,...,...,...,...
40583,D-apeX,5,Wine Town Rollers,United States,"1st June, 2024",http://www.derbyrollcall.com/everyone
40584,BrowSin,137,Texoma Roller Derby,United States,"3rd June, 2024",http://www.derbyrollcall.com/everyone
40585,Daisy Nukes,47,Prison City Roller Derby,United States,"3rd June, 2024",http://www.derbyrollcall.com/everyone
40586,Skidrow sasha,888,Gold coast derby grrls,United States,"5th June, 2024",http://www.derbyrollcall.com/everyone


In [38]:
# Fetch all names beginning with a letter from RollerDerbyRoster
def get_page_names(initial_letter, timeout=30):
    temp_names = []
    url = "https://rollerderbyroster.com/view-names/?ini={}".format(letter)
    # print("Downloading names from {}".format(url))
    try:
        r = session.get(url=url)
        soup = BeautifulSoup(markup=r.text, features="lxml")
        rows = soup.find_all(name="ul")
        # Use only last unordered list - this is where names are!
        for _, li in enumerate(iterable=rows[-1]):
            # Name should be the text of the link within the list item
            name = li.find("a").get_text()
            temp_names.append(name)
    except requests.Timeout:
        print("Timeout!")
        pass
    return temp_names

In [39]:
# Get all names from RollerDerbyRoster
initial_letters = string.ascii_letters + string.digits + string.punctuation
rdr_df = pd.DataFrame()

pbar = tqdm(iterable=initial_letters)
for letter in pbar:
    try:
        pbar.set_description(desc=f"{len(rdr_df)} names found - Processing {letter}")
        temp_names = get_page_names(initial_letter=letter)
        temp_df = pd.DataFrame(data={"Name": temp_names, "url": url})
        rdr_df = pd.concat(objs=[rdr_df, temp_df])
    except Exception as e:
        print(e)
        pass

0 names found - Processing b:   1%|          | 1/94 [00:01<02:39,  1.72s/it]

'int' object has no attribute 'get_text'


0 names found - Processing c:   2%|▏         | 2/94 [00:03<02:19,  1.52s/it]

'int' object has no attribute 'get_text'


0 names found - Processing d:   3%|▎         | 3/94 [00:03<01:41,  1.11s/it]

...

2 names found - Processing /:  82%|████████▏ | 77/94 [02:10<00:04,  4.17it/s]

'int' object has no attribute 'get_text'


2 names found - Processing {:  96%|█████████▌| 90/94 [02:13<00:00,  4.73it/s]

'int' object has no attribute 'get_text'


2 names found - Processing ~: 100%|██████████| 94/94 [02:14<00:00,  1.43s/it]


In [18]:
# If no data was found, load from CSV
if len(rdr_df) == 0:
    rdr_df = pd.read_csv(filepath_or_buffer="rollerderbyroster.csv")
else:
    rdr_df.to_csv(path_or_buf="rollerderbyroster.csv", index=False)
rdr_df

,Name,url
0,A B Old Tricksee,http://www.derbyrollcall.com/everyone
1,A Blonde with No Name,http://www.derbyrollcall.com/everyone
2,A Bomb,http://www.derbyrollcall.com/everyone
3,A Bout To Snap,http://www.derbyrollcall.com/everyone
4,A Boy Named Rob,http://www.derbyrollcall.com/everyone
...,...,...
82825,.50 Cal,http://www.derbyrollcall.com/everyone
82826,.50 Caliber Killer,http://www.derbyrollcall.com/everyone
82827,.50 Kal Kitten,http://www.derbyrollcall.com/everyone
82828,.CC Rip Jaw,http://www.derbyrollcall.com/everyone


In [17]:
# Get RDNation sitemap
rdn_sitemap_url = "https://rdnation.com/sitemap.xml"
rdn_sitemaps = adv.sitemap_to_df(sitemap_url=rdn_sitemap_url)
# League pages have a specific URL structure
rdn_sitemaps["is_league"] = (
    rdn_sitemaps["loc"].str.contains(pat="roller-derby-league/").fillna(False)
)

URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [ ]:
# Extract league URLs
rdn_league_urls = sorted(
    rdn_sitemaps[
        rdn_sitemaps["is_league"]
        & (
            rdn_sitemaps["loc"].str.contains(pat="/2/")
            | rdn_sitemaps["loc"].str.contains(pat="/1/")
        )
    ]["loc"].tolist()
)

In [20]:
rdn_df = pd.DataFrame()


In [ ]:
# Loop through league pages and extract derby names
pbar = tqdm(rdn_league_urls)

for url in pbar:
    try:
        pbar.set_description(desc=f"{len(rdn_df)} names found")
        league_df = pd.concat(objs=pd.read_html(io=url), ignore_index=True)
        league_df.rename(columns={"Derby Name": "Name"}, inplace=True)
        league_df["url"] = url
        rdn_df = pd.concat(objs=[rdn_df, league_df])
    except ValueError as e:
        continue

In [21]:
# If no data was found, load from CSV
if len(rdn_df) == 0:
    rdn_df = pd.read_csv(filepath_or_buffer="rdnation.csv")
else:
    rdn_df.dropna(how="all", axis="columns", inplace=True)
    rdn_df.drop_duplicates(subset=["Name", "Number"], inplace=True)
    rdn_df.sort_values(by="Name", inplace=True)
    rdn_df.to_csv(path_or_buf="rdnation.csv", index=False)
rdn_df

,Name,Number,url
0,!Wolfespit,16,https://rdnation.com/roller-derby-league/2/chi...
1,$3 bill,STAFF,https://rdnation.com/roller-derby-league/1/car...
2,'Lil Diablo,72,https://rdnation.com/roller-derby-league/1/ham...
3,.357 Maggie,6040,https://rdnation.com/roller-derby-league/1/cha...
4,.38 SpeShell,38,https://rdnation.com/roller-derby-league/1/lav...
...,...,...,...
26004,Zulu Mother Smother,360,https://rdnation.com/roller-derby-league/1/m-i...
26005,Zulu Xray,911,https://rdnation.com/roller-derby-league/1/rol...
26006,Zuul,REF,https://rdnation.com/roller-derby-league/1/ora...
26007,Zyklon C,HCN1,https://rdnation.com/roller-derby-league/2/cro...


In [22]:
name_df = pd.concat(objs=[twoevils_df, drc_df, rdr_df, wftda_df, rdn_df], ignore_index=True)
# remove parenthetical phrases from names - eg "(cleared)"
name_df["Name"] = name_df["Name"].str.replace(pat=r"\([^()]*\)", repl="").str.strip()
# remove names with only one character
name_df = name_df.loc[name_df["Name"].str.len() > 1]
name_df

,Name,Number,Registered,League,url,Country
0,!(ED,REF,2012-02-20,Santiago Roller Derby,https://www.twoevils.org/rollergirls/,NaN
1,!Wolfespit,16,2011-02-01,Chilli Padi Derby Grrrls,https://www.twoevils.org/rollergirls/,NaN
2,"""A"" Cup Annihilator",36A,2010-10-13,Billings Roller Derby,https://www.twoevils.org/rollergirls/,NaN
3,"""Bobby"" Val Halen",1984,2011-02-24,Roller Derby Quebec,https://www.twoevils.org/rollergirls/,NaN
4,"""Chupa'clark'bra""",333,2012-01-09,Killa Hurtz Roller Girls,https://www.twoevils.org/rollergirls/,NaN
...,...,...,...,...,...,...
190535,Zulu Mother Smother,360,NaN,NaN,https://rdnation.com/roller-derby-league/1/m-i...,NaN
190536,Zulu Xray,911,NaN,NaN,https://rdnation.com/roller-derby-league/1/rol...,NaN
190537,Zuul,REF,NaN,NaN,https://rdnation.com/roller-derby-league/1/ora...,NaN
190538,Zyklon C,HCN1,NaN,NaN,https://rdnation.com/roller-derby-league/2/cro...,NaN


In [23]:
name_df["Number"].value_counts()

Number
REF       2718
13        1759
11        1110
7         1043
22         990
          ... 
T1NK         1
9116         1
Pi           1
2of          1
128ï¿½       1
Name: count, Length: 11516, dtype: int64

In [24]:
csv_file = Path("derby_names.csv")
name_df.drop_duplicates().sort_values(by=["Name"]).to_csv(path_or_buf=csv_file, index=False)

if "google.colab" in sys.modules:
    files.download(csv_file)

In [25]:
names_only = name_df[["Name"]].drop_duplicates().sort_values(by=["Name"])
names_only.to_csv(path_or_buf="derby_names.txt", index=False, header=False)

if "google.colab" in sys.modules:
    files.download("derby_names.txt")

In [26]:
numbers = name_df[["Number"]].copy()
numbers["Number"] = numbers["Number"].astype(str)
numbers = numbers.sort_values(by=["Number"]).drop_duplicates()
numbers.to_csv(path_or_buf="derby_numbers.txt", index=False, header=False)

if "google.colab" in sys.modules:
    files.download("derby_numbers.txt")

In [27]:
names_numbers = (
    name_df[~name_df["Number"].isna()][["Name", "Number"]]
    .drop_duplicates()
    .sort_values(by=["Name", "Number"])
)
names_numbers.to_csv(path_or_buf="derby_names_numbers.tsv", index=False, header=False, sep="\t")

if "google.colab" in sys.modules:
    files.download("derby_names_numbers.tsv")